In [33]:
import pandas as pd

import tensorflow as tf
from tensorflow import keras

!pip install -q -U keras-tuner
import keras_tuner as kt

from sklearn.feature_selection import RFECV 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.cluster import KMeans
import plotly.express as px
import matplotlib.pyplot as plt



In [2]:
mvp = pd.read_csv("mvp_votings.csv")
nba_df = pd.read_csv("nba_info.csv")
mvp["award_share"]

0      0.658
1      0.613
2      0.414
3      0.261
4      0.120
       ...  
632    0.006
633    0.005
634    0.005
635    0.004
636    0.002
Name: award_share, Length: 637, dtype: float64

In [3]:
print(mvp.columns)
print(nba_df.columns)

Index(['Unnamed: 0', 'fga', 'fg3a', 'fta', 'per', 'ts_pct', 'usg_pct', 'bpm',
       'season', 'player', 'win_pct', 'votes_first', 'points_won',
       'points_max', 'award_share', 'g', 'mp_per_g', 'pts_per_g', 'trb_per_g',
       'ast_per_g', 'stl_per_g', 'blk_per_g', 'fg_pct', 'fg3_pct', 'ft_pct',
       'ws', 'ws_per_48'],
      dtype='object')
Index(['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON',
       'TEAM_ID_home', 'HOME_TEAM_WINS', 'TEAM_ID', 'CONFERENCE', 'TEAM',
       'PLAYER_ID', 'PLAYER_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'REB', 'AST', 'STL', 'BLK',
       'TO', 'PTS'],
      dtype='object')


In [4]:
mvp_groupedII = mvp.groupby(["season", "player"]).mean()
mvp_groupedII.head()

Unnamed: 0   fga  fg3a  fta   per  ts_pct  usg_pct  \
season  player                                                               
1980-81 Adrian Dantley        11.0  20.3   0.1  9.8  24.3   0.622     28.4   
        Artis Gilmore         21.0  10.0   0.0  6.5  21.7   0.699     18.5   
        Bernard King          13.0  15.4   0.1  5.4  19.9   0.617     23.2   
        Bob Lanier            22.0  10.7   0.0  4.1  19.7   0.573     21.8   
        Bobby Jones           26.0   9.3   0.0  4.3  20.1   0.604     20.8   

                        bpm   win_pct  votes_first  ...  pts_per_g  trb_per_g  \
season  player                                      ...                         
1980-81 Adrian Dantley  4.6  0.341463          1.0  ...       30.7        6.4   
        Artis Gilmore   4.9  0.548780          0.0  ...       17.9       10.1   
        Bernard King    3.3  0.475610          0.0  ...       21.9        6.8   
        Bob Lanier      3.4  0.731707          0.0  ...       14.3        6.2   
        Bobby Jones     5.0  0.756098          0.0  ...       13.5        5.4   

                        ast_per_g  stl_per_g  blk_per_g  fg_pct  fg3_pct  \
season  player                                                             
1980-81 Adrian Dantley        4.0        1.4        0.2   0.559    0.286   
        Artis Gilmore         2.1        0.6        2.4   0.670    0.000   
        Bernard King          3.5        0.9        0.4   0.588    0.333   
        Bob Lanier            2.7        1.1        1.2   0.525    1.000   
        Bobby Jones           2.8        1.2        0.9   0.539    0.000   

                        ft_pct    ws  ws_per_48  
season  player                                   
1980-81 Adrian Dantley   0.806  13.6      0.191  
        Artis Gilmore    0.705  12.3      0.208  
        Bernard King     0.703   9.1      0.150  
        Bob Lanier       0.751   6.8      0.185  
        Bobby Jones      0.813   9.2      0.217  

[5 rows x 25 columns]

In [5]:
nba_cleaning = nba_df.drop(columns=['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'TEAM_ID_home', 'HOME_TEAM_WINS', 'TEAM_ID', 'CONFERENCE', 'TEAM'])
nba_cleaning.head()

,GAME_DATE_EST,GAME_ID,SEASON,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,...,FG3_PCT,FTM,FTA,FT_PCT,REB,AST,STL,BLK,TO,PTS
0,2022-12-22,22200477,2022-01-01,1629641,Romeo Langford,18.0,1.0,1.0,1.000,0.0,...,0.000,0.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,2.0
1,2022-12-22,22200477,2022-01-01,1631110,Jeremy Sochan,31.0,7.0,14.0,0.500,2.0,...,0.500,7.0,10.0,0.7,9.0,6.0,1.0,0.0,2.0,23.0
2,2022-12-22,22200477,2022-01-01,1627751,Jakob Poeltl,21.0,6.0,9.0,0.667,0.0,...,0.000,1.0,1.0,1.0,4.0,1.0,1.0,0.0,2.0,13.0
3,2022-12-22,22200477,2022-01-01,1630170,Devin Vassell,30.0,4.0,13.0,0.308,1.0,...,0.167,1.0,1.0,1.0,9.0,5.0,3.0,0.0,2.0,10.0
4,2022-12-22,22200477,2022-01-01,1630200,Tre Jones,27.0,7.0,12.0,0.583,1.0,...,0.333,4.0,4.0,1.0,2.0,3.0,0.0,0.0,2.0,19.0


In [6]:
mvp_groupedII["award_share"]

season   player           
1980-81  Adrian Dantley       0.022
         Artis Gilmore        0.006
         Bernard King         0.017
         Bob Lanier           0.006
         Bobby Jones          0.004
                              ...  
2017-18  LaMarcus Aldridge    0.006
         LeBron James         0.731
         Russell Westbrook    0.075
         Stephen Curry        0.005
         Victor Oladipo       0.002
Name: award_share, Length: 637, dtype: float64

In [7]:
mvp_cat = mvp_groupedII.dtypes[mvp_groupedII.dtypes == "object"].index.tolist()
mvp_groupedII.dtypes

Unnamed: 0     float64
fga            float64
fg3a           float64
fta            float64
per            float64
ts_pct         float64
usg_pct        float64
bpm            float64
win_pct        float64
votes_first    float64
points_won     float64
points_max     float64
award_share    float64
g              float64
mp_per_g       float64
pts_per_g      float64
trb_per_g      float64
ast_per_g      float64
stl_per_g      float64
blk_per_g      float64
fg_pct         float64
fg3_pct        float64
ft_pct         float64
ws             float64
ws_per_48      float64
dtype: object

In [8]:
def get_clusters(k, data) :
# Create a copy of the DataFrame
  data = data.copy()
# Initialize the K-Means model
  model = KMeans(n_clusters=k, random_state=0)
# Fit the model
  model.fit(data)
# Predict clusters
  predictions = model.predict(data)
# Create return DataFrame with predicted clusters
  data["class"] = model.labels_
  return data

In [9]:
two_clusters = get_clusters(6, mvp_groupedII)
two_clusters.head()

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Unnamed: 0   fga  fg3a  fta   per  ts_pct  usg_pct  \
season  player                                                               
1980-81 Adrian Dantley        11.0  20.3   0.1  9.8  24.3   0.622     28.4   
        Artis Gilmore         21.0  10.0   0.0  6.5  21.7   0.699     18.5   
        Bernard King          13.0  15.4   0.1  5.4  19.9   0.617     23.2   
        Bob Lanier            22.0  10.7   0.0  4.1  19.7   0.573     21.8   
        Bobby Jones           26.0   9.3   0.0  4.3  20.1   0.604     20.8   

                        bpm   win_pct  votes_first  ...  trb_per_g  ast_per_g  \
season  player                                      ...                         
1980-81 Adrian Dantley  4.6  0.341463          1.0  ...        6.4        4.0   
        Artis Gilmore   4.9  0.548780          0.0  ...       10.1        2.1   
        Bernard King    3.3  0.475610          0.0  ...        6.8        3.5   
        Bob Lanier      3.4  0.731707          0.0  ...        6.2        2.7   
        Bobby Jones     5.0  0.756098          0.0  ...        5.4        2.8   

                        stl_per_g  blk_per_g  fg_pct  fg3_pct  ft_pct    ws  \
season  player                                                                
1980-81 Adrian Dantley        1.4        0.2   0.559    0.286   0.806  13.6   
        Artis Gilmore         0.6        2.4   0.670    0.000   0.705  12.3   
        Bernard King          0.9        0.4   0.588    0.333   0.703   9.1   
        Bob Lanier            1.1        1.2   0.525    1.000   0.751   6.8   
        Bobby Jones           1.2        0.9   0.539    0.000   0.813   9.2   

                        ws_per_48  class  
season  player                            
1980-81 Adrian Dantley      0.191      2  
        Artis Gilmore       0.208      2  
        Bernard King        0.150      2  
        Bob Lanier          0.185      2  
        Bobby Jones         0.217      2  

[5 rows x 26 columns]

In [10]:
# Verification of different clusters
two_clusters["class"].nunique()

6

In [11]:
!pip install hvplot


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.3 MB/s eta 0:00:00


In [17]:
import hvplot.pandas
!pip install -U bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.4.3
    Uninstalling bokeh-2.4.3:
      Successfully uninstalled bokeh-2.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.14.4 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.1.0 which is incompatible.


In [18]:
%matplotlib inline
two_clusters.hvplot.scatter(x="fga", y="per", by="class")

:NdOverlay   [class]
   :Scatter   [fga]   (per)

In [14]:
two_clusters.dtypes

Unnamed: 0     float64
fga            float64
fg3a           float64
fta            float64
per            float64
ts_pct         float64
usg_pct        float64
bpm            float64
win_pct        float64
votes_first    float64
points_won     float64
points_max     float64
award_share    float64
g              float64
mp_per_g       float64
pts_per_g      float64
trb_per_g      float64
ast_per_g      float64
stl_per_g      float64
blk_per_g      float64
fg_pct         float64
fg3_pct        float64
ft_pct         float64
ws             float64
ws_per_48      float64
class            int32
dtype: object

## THE MODEL

In [19]:
# For demonstration purposes only

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mvp_groupedII[mvp_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(mvp_cat)
encode_df.head()

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


""
0
1
2
3
4


In [22]:
two_clusters.columns

Index(['Unnamed: 0', 'fga', 'fg3a', 'fta', 'per', 'ts_pct', 'usg_pct', 'bpm',
       'win_pct', 'votes_first', 'points_won', 'points_max', 'award_share',
       'g', 'mp_per_g', 'pts_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g',
       'blk_per_g', 'fg_pct', 'fg3_pct', 'ft_pct', 'ws', 'ws_per_48', 'class'],
      dtype='object')

In [29]:
research_df = two_clusters[["award_share", "fga", "per", "ws", "ws_per_48", "usg_pct","ast_per_g", "stl_per_g",
       "blk_per_g"]]
research_df

award_share   fga   per    ws  ws_per_48  usg_pct  \
season  player                                                                 
1980-81 Adrian Dantley           0.022  20.3  24.3  13.6      0.191     28.4   
        Artis Gilmore            0.006  10.0  21.7  12.3      0.208     18.5   
        Bernard King             0.017  15.4  19.9   9.1      0.150     23.2   
        Bob Lanier               0.006  10.7  19.7   6.8      0.185     21.8   
        Bobby Jones              0.004   9.3  20.1   9.2      0.217     20.8   
...                                ...   ...   ...   ...        ...      ...   
2017-18 LaMarcus Aldridge        0.006  18.0  25.0  10.9      0.209     29.1   
        LeBron James             0.731  19.3  28.6  14.0      0.221     31.6   
        Russell Westbrook        0.075  21.1  24.7  10.1      0.166     34.1   
        Stephen Curry            0.005  16.9  28.2   9.1      0.267     31.0   
        Victor Oladipo           0.002  17.9  23.1   8.2      0.155     30.1   

                           ast_per_g  stl_per_g  blk_per_g  
season  player                                              
1980-81 Adrian Dantley           4.0        1.4        0.2  
        Artis Gilmore            2.1        0.6        2.4  
        Bernard King             3.5        0.9        0.4  
        Bob Lanier               2.7        1.1        1.2  
        Bobby Jones              2.8        1.2        0.9  
...                              ...        ...        ...  
2017-18 LaMarcus Aldridge        2.0        0.6        1.2  
        LeBron James             9.1        1.4        0.9  
        Russell Westbrook       10.3        1.8        0.3  
        Stephen Curry            6.1        1.6        0.2  
        Victor Oladipo           4.3        2.4        0.8  

[637 rows x 9 columns]

In [35]:
# Remove loan status target from features data
y = research_df.award_share
X = research_df.drop(columns=["award_share"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
from sklearn import preprocessing
from sklearn import utils

In [42]:
enc = preprocessing.LabelEncoder()
encoded = enc.fit_transform(y_train)
print(utils.multiclass.type_of_target(research_df))

print(utils.multiclass.type_of_target(research_df.astype('int')))

print(utils.multiclass.type_of_target(research_df))


continuous-multioutput
multiclass-multioutput
continuous-multioutput


In [41]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

ValueError: ignored

In [43]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
15/15 [==============================] - 3s 6ms/step - loss: 0.5658 - accuracy: 0.0000e+00
Epoch 2/50
15/15 [==============================] - 0s 3ms/step - loss: 0.5081 - accuracy: 0.0000e+00
Epoch 3/50
15/15 [==============================] - 0s 5ms/step - loss: 0.4647 - accuracy: 0.0000e+00
Epoch 4/50
15/15 [==============================] - 0s 3ms/step - loss: 0.4294 - accuracy: 0.0000e+00
Epoch 5/50
15/15 [==============================] - 0s 4ms/step - loss: 0.4007 - accuracy: 0.0000e+00
Epoch 6/50
15/15 [==============================] - 0s 11ms/step - loss: 0.3787 - accuracy: 0.0000e+00
Epoch 7/50
15/15 [==============================] - 0s 11ms/step - loss: 0.3617 - accuracy: 0.0021
Epoch 8/50
15/15 [==============================] - 0s 6ms/step - loss: 0.3493 - accuracy: 0.0021
Epoch 9/50
15/15 [==============================] - 0s 4ms/step - loss: 0.3401 - accuracy: 0.0021
Epoch 10/50
15/15 [==============================] - 0s 5ms/step - loss: 0.3332 - accuracy: 